In [1]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [2]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_md_testing/tune_logs/MGNN_hyp_small_full_mat_loss_md.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        trials.append((params, eval_res))

In [3]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
for trial in trials: 
    print(trial[0])
    print(trial[1]["cummulative_stats"])
    print("---")


{'batch_size': 8, 'data_aug_factor': 4, 'edge_threshold_val': 3.4729546323381877, 'grace_epochs': 5, 'hidden_dim': 512, 'loss_on_full_matrix': True, 'lr': 0.00026609608815425317, 'lr_cooldown': 2, 'lr_factor': 0.5, 'lr_min': 1e-06, 'lr_patience': 3, 'lr_threshold': 0.001, 'message_net_dropout': 0.003298343626941691, 'message_net_layers': 4, 'message_passing_steps': 2, 'num_epochs': 30, 'weight_decay': 0.00017118995475258337}
{'energy_abs': {'mean': -1243.924432159559, 'std': 38.445296744325056}, 'iterations': {'mean': 11.476190476190476, 'std': 0.9571310115353228}, 'energy_rel': {'mean': -0.7132488533198056, 'std': 0.010936525690007678}, 'diis': {'mean': 0.20330397955926433, 'std': 0.016186910136091078}, 'rmse': {'mean': 0.007402123613588803, 'std': 0.0006294361313717451}}
---
{'batch_size': 8, 'data_aug_factor': 4, 'edge_threshold_val': 3.27252771373414, 'grace_epochs': 5, 'hidden_dim': 128, 'loss_on_full_matrix': True, 'lr': 0.00034665740607485115, 'lr_cooldown': 2, 'lr_factor': 0.5,

In [7]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.2f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 11.5(1.0) & -1243.9(38.4) & -0.713(0.011) & 0.203(0.02) & 0.0074(0.0006) \\
1 & 11.5(0.7) & -1252.8(37.7) & -0.717(0.011) & 0.211(0.02) & 0.0076(0.0006) \\
2 & 11.6(0.8) & -1253.7(37.5) & -0.718(0.010) & 0.208(0.02) & 0.0076(0.0006) \\
3 & 11.7(1.0) & -1256.3(38.5) & -0.719(0.012) & 0.211(0.02) & 0.0075(0.0006) \\
4 & 11.9(0.9) & -1257.4(39.2) & -0.720(0.012) & 0.214(0.02) & 0.0085(0.0007) \\
5 & 12.3(0.9) & -1287.5(37.2) & -0.737(0.011) & 0.224(0.02) & 0.0086(0.0005) \\
6 & 12.3(1.0) & -1282.2(35.7) & -0.734(0.010) & 0.222(0.02) & 0.0083(0.0006) \\
7 & 12.5(1.0) & -1290.1(39.7) & -0.739(0.011) & 0.222(0.02) & 0.0084(0.0006) \\
8 & 12.5(1.2) & -1280.0(42.8) & -0.733(0.013) & 0.221(0.02) & 0.0091(0.0016) \\
9 & 12.5(1.1) & -1293.1(41.2) & -0.740(0.013) & 0.223(0.02) & 0.0088(0.0006) \\
\bottomrule
\end{tabular}



In [8]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 512 & 128 & 128 & 256 & 512 & 256 & 128 & 512 & 512 & 512 \\
Batch Size & 8 & 8 & 8 & 8 & 32 & 32 & 32 & 16 & 32 & 32 \\
Data aug. & 4.00 & 4.00 & 3.00 & 3.00 & 2.00 & 2.00 & 3.00 & 4.00 & 3.00 & 3.00 \\
Edge thres. & 3.47 & 3.27 & 3.48 & 2.79 & 2.72 & 3.30 & 2.69 & 3.67 & 3.68 & 2.25 \\
Message p. steps & 2 & 4 & 4 & 2 & 2 & 4 & 3 & 4 & 3 & 5 \\
Message Net Dropout & 0.00 & 0.06 & 0.27 & 0.05 & 0.11 & 0.10 & 0.14 & 0.19 & 0.15 & 0.18 \\
Message Net Layers & 4 & 3 & 2 & 3 & 3 & 4 & 3 & 4 & 4 & 2 \\
Learn Rate & 2.66e-04 & 3.47e-04 & 8.28e-04 & 1.28e-04 & 1.00e-04 & 5.09e-04 & 2.51e-03 & 8.23e-04 & 1.78e-03 & 1.94e-03 \\
Weight Decay & 1.71e-04 & 2.57e-05 & 1.60e-05 & 3.38e-04 & 3.04e-04 & 9.69e-04 & 5.79e-04 & 4.80e-05 & 5.27e-05 & 1.37e-05 \\
\bottomrule
\end{tabular}

